In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!pip install git+https://github.com/michelecafagna26/CLIPCap.git


  Cloning https://github.com/michelecafagna26/CLIPCap.git to /private/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/pip-req-build-5a_7e3o1
  Running command git clone --filter=blob:none --quiet https://github.com/michelecafagna26/CLIPCap.git /private/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/pip-req-build-5a_7e3o1
  Resolved https://github.com/michelecafagna26/CLIPCap.git to commit 545c5574fee8e3a56dec3e5c3f86388269a06f1d
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP (to revision main) to /private/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/pip-install-ri62hat6/clip_dbaadd16ea194a03a22e082ff52f152c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP /private/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/pip-install-ri62hat6/clip_dbaadd16ea194a03a22e082ff52f152c
  Resolved https://github.com/openai/CLIP to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Using ca

In [ ]:
git clone https://github.com/rmokady/CLIP_prefix_caption && cd CLIP_prefix_caption
conda env create -f environment.yml
conda activate clip_prefix_caption

## 데이터셋을 pkl 형식으로 만들어야 합니다

In [ ]:
!python /content/drive/MyDrive/MLB/NIPA/CLIP_prefix_caption/parse_coco.py --clip_model_type ViT-B/32

## TRAIN

공식 문서 참고해서 파인튜닝 할 것


epochs = 10 
필요시 train.py에서 바꿀것 
batch_size = 40 

In [ ]:
!python /content/drive/MyDrive/MLB/NIPA/CLIP_prefix_caption/train.py --data /content/drive/MyDrive/MLB/NIPA/CLIP_prefix_caption_ViT-B_32_train.pkl --out_dir /content/drive/MyDrive/MLB/NIPA/CLIP_prefix_caption_ViT-B_32_train_out --pretrained_model /content/drive/MyDrive/MLB/NIPA/CLIP_prefix_caption/model_wieghts.pt

## library

In [82]:
from clipcap import ClipCaptionModel
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
)
import torch
import clip
import requests
from PIL import Image
import os 
import json 

In [ ]:
# load pretrained model
def load_pretrained_model(model, pretrained_path):
    if os.path.isfile(pretrained_path):
        print(f"Loading pre-trained model from {pretrained_path}")
        # strict=False를 추가하여 일부 매개변수가 없어도 로딩 가능하도록 설정
        model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')), strict=False)
        print("Model loaded successfully.")
    else:
        print(f"Pre-trained model file {pretrained_path} not found.")
    return model


## pretrained model by openAI

In [85]:
# load pretrained model
def load_pretrained_model(model, pretrained_path):
    if os.path.isfile(pretrained_path):
        print(f"Loading pre-trained model from {pretrained_path}")
        # strict=False를 추가하여 일부 매개변수가 없어도 로딩 가능하도록 설정
        model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')), strict=False)
        print("Model loaded successfully.")
    else:
        print(f"Pre-trained model file {pretrained_path} not found.")
    return model

#model_path = "/Users/psjj/Downloads/coco_prefix-009.pt"  # change accordingly
model_path = '/Users/psjj/Downloads/model_wieghts.pt'
# load clip
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
prefix_length = 10

# load ClipCap
model = ClipCaptionModel(prefix_length, tokenizer=tokenizer)

# load the pretrained model weights
model = load_pretrained_model(model, model_path)
model = model.eval()
model = model.to(device)

# load the image
image_dir = '/Users/psjj/Downloads/coco2017/test2017/000000000019.jpg'
raw_image = Image.open(image_dir).convert('RGB')

# extract the prefix
image = preprocess(raw_image).unsqueeze(0).to(device)
with torch.no_grad():
    prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
    prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

# assuming generate_beam() is correctly implemented in ClipCaptionModel
caption = model.generate_beam(embed=prefix_embed)[0]

print("Generated caption:", caption)


/opt/anaconda3/envs/NIPA/lib/python3.11/site-packages/transformers/modeling_utils.py:1435: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(resolved_arc

Loading pre-trained model from /Users/psjj/Downloads/model_wieghts.pt


/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/ipykernel_98814/4285155122.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(pretrained_p

Model loaded successfully.
Generated caption: a couple of cows are laying in a field together 
 
 
                                                   


## inference

In [86]:
# load pretrained model
def load_pretrained_model(model, pretrained_path):
    if os.path.isfile(pretrained_path):
        print(f"Loading pre-trained model from {pretrained_path}")
        # strict=False를 추가하여 일부 매개변수가 없어도 로딩 가능하도록 설정
        model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu')), strict=False)
        print("Model loaded successfully.")
    else:
        print(f"Pre-trained model file {pretrained_path} not found.")
    return model

model_path = "/Users/psjj/Downloads/coco_prefix-009.pt"  # change accordingly
#model_path = '/Users/psjj/Downloads/model_wieghts.pt'
# load clip
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
prefix_length = 10

# load ClipCap
model = ClipCaptionModel(prefix_length, tokenizer=tokenizer)

# load the pretrained model weights
model = load_pretrained_model(model, model_path)
model = model.eval()
model = model.to(device)

# load the image
image_dir = '/Users/psjj/Downloads/coco2017/test2017/000000000019.jpg' # your_path
raw_image = Image.open(image_dir).convert('RGB')

# extract the prefix
image = preprocess(raw_image).unsqueeze(0).to(device)
with torch.no_grad():
    prefix = clip_model.encode_image(image).to(device, dtype=torch.float32)
    prefix_embed = model.clip_project(prefix).reshape(1, prefix_length, -1)

# assuming generate_beam() is correctly implemented in ClipCaptionModel
caption = model.generate_beam(embed=prefix_embed)[0]

print("Generated caption:", caption)


Loading pre-trained model from /Users/psjj/Downloads/coco_prefix-009.pt


/var/folders/xz/3gsy8h1n2mq8_tmb5ll9f2j80000gn/T/ipykernel_98814/3215760643.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(pretrained_p

Model loaded successfully.
Generated caption: 

The following is a list of the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common, and the most common,
